In [28]:
import pandas as pd
import re

def convert_formats_to_floats(df):
    pattern = re.compile(r'^-?\d{1,3}(?:,\d{3})*(?:\.\d+)?$')
    df = df.apply(lambda x: x.str.rstrip('%').str.replace(',', '').astype('float') / 100 
                              if x.dtype == object and '%' in x.iloc[0] 
                              else x)
    df = df.apply(lambda x: x.apply(lambda y: float(y) if pattern.match(str(y)) else y) 
                          if x.dtype == object else x)
    return df

In [29]:
import re

pattern = re.compile(r'(\((\d{1,3}(?:,\d{3})*|\d+(?:\.\d+)?)\)|-?\d{1,3}(?:,\d{3})*(?:\.\d+)?)')

my_string = '-1248.87'

if pattern.search(my_string):
    print(f"match found: {my_string}")
    print(pattern.search(my_string).group())
else:
    print('fail')


match found: -1248.87
-124


In [ ]:
import pandas as pd
import re

def convert_percent(value):
    return float(value.replace('%', '')) / 100

def convert_dollar(value):
    return float(value.replace('$', '').replace(',', ''))

def convert_accounting(value):
    return -1 * float(value.replace('(', '-').replace(')', '').replace('$', '').replace(',', ''))

def convert_number_formats(df):
    regex_compilers = {
        'percent': re.compile(r'\d+(\.\d+)?%'),
        'dollar': re.compile(r'\$[\d,]+(\.\d+)?'),
        'accounting': re.compile(r'\(\$\d+([\d,]+)?(\.\d+)?\)')
    }

    for col in df.columns:
        if df[col].dtype == 'object':
            for format, regex in regex_compilers.items():
                mask = df[col].str.contains(regex)
                if mask.any():
                    if format == 'percent':
                        df.loc[mask, col] = df.loc[mask, col].apply(lambda x: convert_percent(x))
                    elif format == 'dollar':
                        df.loc[mask, col] = df.loc[mask, col].apply(lambda x: convert_dollar(x))
                    elif format == 'accounting':
                        df.loc[mask, col] = df.loc[mask, col].apply(lambda x: convert_accounting(x))
    
    return df

# Create an example DataFrame
df = pd.DataFrame({'A': ['10%', '$1,234.56', '(5,000)', 'N/A'], 'B': ['(3,000.50)', '$500', '2,000.50', 'N/A']})

# Convert number formats in the DataFrame
df = convert_number_formats(df)

print(df)


In [83]:
import time

def measure_runtime(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        runtime = end_time - start_time
        print(f"Runtime of {func.__name__}: {runtime} seconds")
        return result
    return wrapper


In [90]:
import numpy as np
import pandas as pd

# method 1
def find_digit(str_value: str) -> float:
    temp_bin = []
    temp_values = [1,1]
    for ch in str_value:
        if '-' in ch or '.' in ch:
            temp_bin.append(ch)
        elif ch.isdigit():
            temp_bin.append(ch)
        elif ch == "(":
            temp_values[0] = -1
        elif ch == "%":
            temp_values[1] = 100
    new_value = ''.join(temp_bin)
    if new_value != '':
        return (float(new_value) * temp_values[0] ) / temp_values[1]
    
    else: return str_value

# method 2
def find_digit(str_value: str) -> float:
    if re.search('[a-zA-Z]', str_value):
        return str_value
    temp_bin = [ch for ch in str_value if ch.isdigit() or ch in '-.']
    temp_values = [1, 1]
    
    for ch in str_value:
        if ch == '(':
            temp_values[0] = -1
        elif ch == '%':
            temp_values[1] = 100
    
    new_value = ''.join(temp_bin)
    if new_value != '':
        return (float(new_value) * temp_values[0]) / temp_values[1]
    else:
        return str_value

@measure_runtime
def convert_number_formats(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: find_digit(x) if pd.notnull(x) else x)
    return df

# Create an example DataFrame
df = pd.DataFrame({'A': ['10%', '$1,234.56', '(5,000)', 'N/A'], 'B': ['(3,0000.50)', '$500', '2,000.50', 'N/A']})

# Convert number formats in the DataFrame
df = convert_number_formats(df)

df.replace(['N/A', 'None', ''], np.nan, inplace=True)

print(df)


Runtime of convert_number_formats: 0.0019986629486083984 seconds
         A        B
0     0.10 -30000.5
1  1234.56    500.0
2 -5000.00   2000.5
3      NaN      NaN


In [92]:
method_1 = 0.0010008811950683594
method_2 = 0.0019881725311279297

(method_1 - method_2) / method_1

-0.9864221057646498

In [66]:
df.dtypes

A    float64
B    float64
dtype: object